In [9]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_validate
import numpy as np
import matplotlib.pyplot as  plt
%matplotlib inline

defenders = pd.read_csv(r'C:\Users\sosan\Documents\Dissertation\DataSets\Fifa 20\TransformedData\Defenders.csv')
midfielders = pd.read_csv(r'C:\Users\sosan\Documents\Dissertation\DataSets\Fifa 20\TransformedData\Midfielders.csv')
fowards = pd.read_csv(r'C:\Users\sosan\Documents\Dissertation\DataSets\Fifa 20\TransformedData\Strikers.csv')
defenders.head(5)

,Name,Image,Country,Position,Age,Overall,Potential,Club,ID,Height,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,Virgil van Dijk,https://cdn.sofifa.org/players/4/20/203376.png,Netherlands,CB,27,90,90,Liverpool,203376,193.0,...,Medium,Medium,3,77,60,70,71,90,86,425
1,Kalidou Koulibaly,https://cdn.sofifa.org/players/4/20/201024.png,Senegal,CB,28,89,89,Napoli,201024,188.0,...,Medium,High,3,71,28,54,67,89,87,303
2,Sergio Ramos García,https://cdn.sofifa.org/players/4/20/155862.png,Spain,CB,33,89,89,Real Madrid,155862,182.9,...,High,Medium,4,72,68,75,73,87,85,211
3,Giorgio Chiellini,https://cdn.sofifa.org/players/4/20/138956.png,Italy,CB,34,89,89,Juventus,138956,188.0,...,Medium,High,4,68,46,58,60,90,82,106
4,Diego Godín,https://cdn.sofifa.org/players/4/20/182493.png,Uruguay,CB,33,88,88,Inter,182493,188.0,...,Medium,High,3,60,48,64,61,89,83,103


In [10]:
def dropFeatures(data):
    new_data =data.drop(['Image','ID','A/W','D/W','Release Clause','Hits','Joined', 'Base Stats','Total Stats','Loan Date End', 'Release Clause','Position','Wage','Value','Name', 'BOV', 'Potential','Club'], axis = 1)
    return new_data

defenders = dropFeatures(defenders)
midfielders = dropFeatures(midfielders)
fowards = dropFeatures(fowards)

In [11]:
def convertFoot(playerData):
    playerData.loc[playerData['foot'] == 'Right', 'foot'] = 1
    playerData.loc[playerData['foot'] == 'Left', 'foot'] = 0
    return playerData

def convertDefenceLabels(data):
    data.loc[data['BP'] == 'CB', 'BP'] = 0
    data.loc[(data['BP'] == 'LB') | (data['BP'] == 'LWB'), 'BP'] = 1
    data.loc[(data['BP'] == 'RB') | (data['BP'] == 'RWB'), 'BP'] = 2
    return data

def convertFowardLabels(data):
    data.loc[data['BP'] == 'LW', 'BP'] = 0
    data.loc[data['BP'] == 'RW', 'BP'] = 1
    data.loc[(data['BP'] == 'ST') | (data['BP'] == 'CF'), 'BP'] = 2
    return data

def convertMidfieldLabels(data):
    data.loc[data['BP'] == 'CM', 'BP'] = 1
    data.loc[data['BP'] == 'CDM', 'BP'] = 0
    data.loc[data['BP'] == 'CAM', 'BP'] = 2
    data.loc[data['BP'] == 'RM' , 'BP'] = 4
    data.loc[data['BP'] == 'LM', 'BP'] = 3
    return data

In [12]:
#get a list of the numeric features in the dataset
defenders = convertDefenceLabels(defenders)
defenders = convertFoot(defenders)
midfielders = convertMidfieldLabels(midfielders)
midfielders = convertFoot(midfielders)
fowards = convertFowardLabels(fowards)
fowards = convertFoot(fowards)
fowards.head(5)

,Country,Age,Overall,Height,Weight,foot,BP,Growth,Attacking,Crossing,...,GK Reflexes,W/F,SM,IR,PAC,SHO,PAS,DRI,DEF,PHY
0,Portugal,34,93,188.0,83.2,1,2,0,437,84,...,11,4,5,5,90,93,82,89,35,78
1,Egypt,27,90,175.3,71.4,0,0,0,391,79,...,14,3,4,3,93,86,81,89,45,74
2,France,20,89,177.8,73.2,1,1,6,405,78,...,6,4,5,3,96,84,78,90,39,75
3,England,25,89,188.0,89.1,1,2,2,421,75,...,11,4,3,3,70,91,79,81,47,83
4,Poland,30,89,182.9,80.0,1,2,0,405,62,...,10,4,4,4,77,87,74,85,41,82


In [ ]:
def normalizeData(data):
    min_max = MinMaxScaler()
    data[player_features] =  min_max.fit_transform(data[player_features])
    #df_scaled = pd.DataFrame(player, columns=numerical_Data.columns,index=numerical_Data.index)
    return data

In [ ]:
defenders = normalizeData(defenders)
midfielders = normalizeData(midfielders)
fowards = normalizeData(fowards)

In [13]:
#Get a count of Nationalities in the Dataset, make of list of those with over 250 Players (our Major Nations)
nat_counts = defenders.Country.value_counts()
nat_list = nat_counts[nat_counts > 250].index.tolist()

#Replace Nationality with a binary indicator variable for 'Major Nation'
def major_nation(df):
    if (df.Country in nat_list):
        return 1
    else:
        return 0

#Create a copy of the original dataframe to avoid indexing errors
defenders1 = defenders.copy()
midfielders1 = midfielders.copy()
fowards1 = fowards.copy()

defenders1['Major_Nation'] = defenders1.apply(major_nation,axis = 1)
midfielders1['Major_Nation'] = midfielders1.apply(major_nation,axis = 1)
fowards1['Major_Nation'] = fowards1.apply(major_nation,axis = 1)

#Drop original columns used
defenders1 = defenders1.drop(['Country'], axis = 1)
midfielders1 = midfielders1.drop(['Country'], axis = 1)
fowards1 = fowards1.drop(['Country'], axis = 1)
fowards1.head()



,Age,Overall,Height,Weight,foot,BP,Growth,Attacking,Crossing,Finishing,...,W/F,SM,IR,PAC,SHO,PAS,DRI,DEF,PHY,Major_Nation
0,34,93,188.0,83.2,1,2,0,437,84,94,...,4,5,5,90,93,82,89,35,78,0
1,27,90,175.3,71.4,0,0,0,391,79,90,...,3,4,3,93,86,81,89,45,74,0
2,20,89,177.8,73.2,1,1,6,405,78,89,...,4,5,3,96,84,78,90,39,75,1
3,25,89,188.0,89.1,1,2,2,421,75,94,...,4,3,3,70,91,79,81,47,83,1
4,30,89,182.9,80.0,1,2,0,405,62,88,...,4,4,4,77,87,74,85,41,82,0


In [14]:
#Split ID as a Target value
target = defenders1.Overall
explanatory_var = defenders1.drop(['Overall'], axis = 1)

#Splitting into test and train
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(explanatory_var, target, test_size=0.2)

#new flow

dictionaryOfTrain_Test = {}
x_train, x_test, y_train, y_test = train_test_split( defenders1.drop(['Overall'], axis = 1), defenders1.Overall, test_size=0.2)
dictionaryOfTrain_Test['defenders'] = {'x_train': x_train,  'x_test' : x_test, 'y_train':y_train, 'y_test': y_test}

x_train, x_test, y_train, y_test = train_test_split( midfielders1.drop(['Overall'], axis = 1), midfielders1.Overall, test_size=0.2)
dictionaryOfTrain_Test['midfielders'] = {'x_train': x_train,  'x_test' : x_test, 'y_train':y_train, 'y_test': y_test}

x_train, x_test, y_train, y_test = train_test_split( fowards1.drop(['Overall'], axis = 1), fowards1.Overall, test_size=0.2)
dictionaryOfTrain_Test['fowards'] = {'x_train': x_train,  'x_test' : x_test, 'y_train':y_train, 'y_test': y_test}

#One Hot Encoding
#x_train = pd.get_dummies(x_train)
#x_test = pd.get_dummies(x_test)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(3324, 57) (3324,)
(831, 57) (831,)


In [15]:
#Applying Linear Regression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor



#Finding the r2 score and root mean squared error
from sklearn.metrics import r2_score, mean_squared_error

#import plotter
import seaborn as sns

import matplotlib.pyplot as  plt
%matplotlib inline

def applyModel(model, x_data, y_true, pltname):
    y_pred = model.predict(x_data)

    print('\n\nModel Name: ',type(model))
    print(y_pred[:10])
    print(y_true[:10])

    print('r2 score: '+str(round(r2_score(y_true, y_pred),2)))
    print('RMSE : '+str(round(np.sqrt(mean_squared_error(y_true, y_pred)),2)))

    # # get importance
    # importance = model.coef_
    # # summarize feature importance
    # for i,v in enumerate(importance):
    #     print('Feature: %0d, Score: %.5f' % (i,v))
    #     # plot feature importance
    # plt.bar([x for x in range(len(importance))], importance)
    # plt.show()

    #Visualising the results
    # plt.figure()
    # plt.figure(figsize=(18,10))
    # sns.regplot(y_pred, y_true,scatter_kws={'color':'red','edgecolor':'blue','linewidth':'0.7'},line_kws={'color':'black','alpha':0.5})
    # plt.title("Linear Prediction of Player Rating")
    # plt.show()
    #plt.figure(figsize=(18,10))
    # sns.set(color_codes=True)
    # sns.set(rc={'figure.figsize':(18, 10)})
    #sns.regplot(x=y_pred, y=y_true, fit_reg=False,scatter_kws={'color':'red','edgecolor':'blue','linewidth': 0.7},line_kws={'color':'black','alpha':0.5});
    #plt.savefig(pltname + '.png')
    #sns.regplot(x=y_pred, y=y_true,scatter=False, ax=ax, scatter_kws={"color": "green"});

In [16]:
reg_mdels = [LinearRegression(), MLPRegressor(random_state=1, max_iter=500, hidden_layer_sizes = (25, 11, 7))]

positions = ['defenders','midfielders', 'fowards']
for key,value in dictionaryOfTrain_Test.items():
    print('for the ',key)
    loop = 0
    for model in reg_mdels:
        model.fit(value['x_train'], value['y_train'])
        if loop == 0:
            pltname = key + 'LinearaReg'
        else:
            pltname = key + 'MLP'
        applyModel(model, value['x_train'], value['y_train'], pltname)
        loop = loop + 1

for the  defenders


Model Name:  <class 'sklearn.linear_model._base.LinearRegression'>
[62.59986037 68.62042888 73.91170186 65.85382867 65.08692722 67.44457193
 62.07608153 69.95728492 61.10397024 66.54429056]
3774    64
1807    70
494     75
3306    66
3678    65
2653    67
4015    64
1795    70
5196    60
3077    66
Name: Overall, dtype: int64
r2 score: 0.97
RMSE : 1.08


Model Name:  <class 'sklearn.neural_network._multilayer_perceptron.MLPRegressor'>
[62.99351819 68.22548112 74.52141422 65.0818682  64.92672533 66.79321637
 63.26677061 68.77891473 59.35713707 65.67088922]
3774    64
1807    70
494     75
3306    66
3678    65
2653    67
4015    64
1795    70
5196    60
3077    66
Name: Overall, dtype: int64
r2 score: 0.98
RMSE : 0.92
for the  midfielders


Model Name:  <class 'sklearn.linear_model._base.LinearRegression'>
[76.35312346 61.43712166 62.8215771  59.33658996 67.08366492 77.63538137
 69.56805816 79.55938008 61.30038467 61.68918684]
441     77
3663    65
4511    63
5444  

In [18]:
from mlxtend.evaluate import paired_ttest_5x2cv

t, p = paired_ttest_5x2cv(estimator1=reg_mdels[0],
                          estimator2=reg_mdels[1],
                          X=explanatory_var, y=target,
                          random_seed=1)

print('t statistic: %.3f' % t)
print('p value: %.3f' % p)

t statistic: -2.643
p value: 0.046


In [ ]:
reg_mdels = [LinearRegression(), MLPRegressor(random_state=1, max_iter=500, hidden_layer_sizes = (25, 11, 7))]

positions = ['defenders','midfielders', 'fowards']
for key,value in dictionaryOfTrain_Test.items():
    print('for the ',key)
    loop = 0
    for model in reg_mdels:
        model.fit(value['x_train'], value['y_train'])
        if loop == 0:
            pltname = key + 'LinearaReg'
        else:
            pltname = key + 'MLP'
        applyModel(model, value['x_train'], value['y_train'], pltname)

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(reg_mdels[1], random_state=1).fit(dictionaryOfTrain_Test['fowards']['x_test'], dictionaryOfTrain_Test['fowards']['y_test'])
eli5.show_weights(perm, feature_names = dictionaryOfTrain_Test['fowards']['x_test'].columns.tolist())

# for key,value in dictionaryOfTrain_Test.items():
#     print('for the ',key)
#     loop = 0
#     for model in reg_mdels:
#         perm = PermutationImportance(model, random_state=1).fit(value['x_test'], value['y_test'])
#         eli5.show_weights(perm, feature_names = value['x_test'].columns.tolist())

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.inspection import permutation_importance

rf_reg_model = RandomForestRegressor(random_state=1, max_depth=10)
sgd_reg_model = SGDRegressor(random_state=1, max_iter=10)

rf_reg_model.fit(x_train, y_train)
sgd_reg_model.fit(x_train, y_train)

important_feat = rf_reg_model.feature_importances_

sgd_reg_model.fit(x_train, y_train)
# perform permutation importance
results = permutation_importance(sgd_reg_model, x_train, y_train, scoring='neg_mean_squared_error')

# get importance
importance = results.importances_mean.argsort()

print(enumerate(importance))

for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

indices = np.argsort(important_feat)[-15:]  # top 10 features

print(indices)

In [ ]:
indexes = defenders1._get_numeric_data().columns
print(indexes)

In [ ]:
defending_features = ['Age', 'Height', 'Weight', 'Growth', 'Attacking', 'Crossing',
       'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill',
       'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control',
       'Movement', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions',
       'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength',
       'Long Shots', 'Mentality', 'Aggression', 'Interceptions', 'Positioning',
       'Vision', 'Penalties', 'Composure', 'Defending', 'Marking',
       'Standing Tackle', 'Sliding Tackle', 'Goalkeeping', 'GK Diving',
       'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'W/F', 'SM', 'IR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF',
       'PHY']
normalize_data = defenders1

In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max = MinMaxScaler()
normalize_data[defending_features] =  min_max.fit_transform(normalize_data[defending_features])
#df_scaled = pd.DataFrame(player, columns=numerical_Data.columns,index=numerical_Data.index)
normalize_data.head(5)

In [ ]:
#Split ID as a Target value
n_target = normalize_data.Overall
n_explanatory_var = normalize_data.drop(['Overall'], axis = 1)

#Splitting into test and train
from sklearn.model_selection import train_test_split
n_x_train, n_x_test, n_y_train, n_y_test = train_test_split(n_explanatory_var, n_target, test_size=0.2)

#One Hot Encoding
#x_train = pd.get_dummies(x_train)
#x_test = pd.get_dummies(x_test)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
print(n_y_test.head(10))

In [ ]:
#Applying Linear Regression
from sklearn.linear_model import LinearRegression
n_model = LinearRegression()
n_model.fit(n_x_train, n_y_train)
n_predictions = n_model.predict(n_x_test)

print(n_predictions[:10])
print(n_y_test[:10])
#Finding the r2 score and root mean squared error
from sklearn.metrics import r2_score, mean_squared_error
print('r2 score: '+str(r2_score(n_y_test, n_predictions)))
print('RMSE : '+str(np.sqrt(mean_squared_error(n_y_test, n_predictions))))

In [ ]:
n_perm = PermutationImportance(n_model, random_state=1).fit(n_x_test, n_y_test)
eli5.show_weights(n_perm, feature_names = n_x_test.columns.tolist())

In [ ]:
rf_reg_model = RandomForestRegressor(random_state=1, max_depth=10)
sgd_reg_model = SGDRegressor(random_state=1, max_iter=10)

rf_reg_model.fit(n_x_train, n_y_train)
sgd_reg_model.fit(n_x_train, n_y_train)

important_feat = rf_reg_model.feature_importances_

sgd_reg_model.fit(x_train, y_train)
# perform permutation importance
results = permutation_importance(sgd_reg_model, n_x_train, n_y_train, scoring='neg_mean_squared_error')

# get importance
importance = results.importances_mean.argsort()

print(importance)

for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

indices = np.argsort(important_feat)[-15:]  # top 10 features

print(indices)

In [ ]:
plt.title('Feature Importances')
plt.barh(range(len(indices)), important_feat[indices], color='b', align='center')
plt.yticks(range(len(indices)), [n_x_test.columns[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
x_fitted_variable = normalize_data[n_x_test.columns[indices]]
x_fitted_variable.iloc[:5]

In [ ]:
x_fit_train, x_fit_test, y_fit_train, y_fit_test = train_test_split(x_fitted_variable, n_target, test_size=0.2)
print (x_fit_train.shape, y_fit_train.shape)
print (x_fit_test.shape, y_fit_test.shape)

In [ ]:
for model in reg_mdels:
    model.fit(x_fit_train, y_fit_train)
    applyModel(model, x_fit_train, y_fit_train)

In [ ]:
for model in reg_mdels:
    model.fit(x_fit_train, y_fit_train)
    applyModel(model, x_fit_test, y_fit_test)